In [23]:
import pandas as pd
import Reports

TypeError: 'type' object is not subscriptable

In [ ]:
report_list = Reports.get_all_reports()

In [2]:
len(report_list)

10214

In [3]:
import json

def read_normalized_file(filename: str) :
    # NOTE: each file represents an ESG report, and each line represents a page. 
    # In a page, each dict represents a block, which could be a text block or an image block.
    # The images are all deleted. Each block tells the size, font, and location, of a text.
    with open(filename, encoding="utf8") as json_file:
        json_list = list(json_file)
    
    return json_list
        
summ_json = read_normalized_file('/scratch/users/k21148846/report_summary_extracted_data.jsonl')

In [5]:
json.loads(summ_json[100])['reports'][0]

{'original_file_path': 'full/5af136d35bf8ab9cc6116a3498f2a0a943d54c27.pdf',
 'to_name': '5af136d35bf8ab9cc6116a3498f2a0a943d54c27',
 'url': 'https://www.responsibilityreports.com/HostedData/ResponsibilityReportArchive/w/NASDAQ_WNFM_2019.pdf',
 'url_depth': 1,
 'date': '2019',
 'title': '2019 Corporate Sustainability Report',
 'content_type': '',
 'content_disposition': '',
 'content_disposition_file_name': '',
 'upper_url': 'https://www.responsibilityreports.com/Company/wayne-farms-inc',
 'metadata': {'format': 'PDF 1.5',
  'title': '',
  'author': '',
  'subject': '',
  'keywords': '',
  'creator': 'Adobe InDesign 14.0 (Macintosh)',
  'producer': 'Adobe PDF Library 15.0',
  'creationDate': "D:20191121164258-05'00'",
  'modDate': "D:20191121164422-05'00'",
  'trapped': '',
  'encryption': None},
 'source_id': '',
 'contain_table_of_content': False,
 'contain_links': False}

In [9]:
test_json = read_normalized_file('/scratch/users/k21148846/' + 'normalization_text/5af136d35bf8ab9cc6116a3498f2a0a943d54c27' + '.jsonl')

In [ ]:
title_dict_counter: dict[str, int] = {}
for report in report_list:
    for word in report.title.split(' '):
        if not word.isalpha():
            continue
        if word not in title_dict_counter:
            title_dict_counter[word] = 1
        else:
            title_dict_counter[word] += 1

In [ ]:
ds = pd.DataFrame(title_dict_counter.items(), columns=['word', 'count'])
ds = ds.sort_values(by='count', ascending=False)
words_to_remove = ['Report', 'and', 'Annual', 'Progress', 'Development', 'Review', 'Impact', 'Business', 'Activity', 'Summary', 'Statement', 'Data', 'Update', 'Supplement', 'on', 'Reporting']
ds_filtered = ds[~ds['word'].isin(words_to_remove)].head(20)

In [ ]:
ds_filtered['count'].sum()

In [ ]:
in_title_counter = 0
not_found_reports_list = []
is_found_word = False

for report in report_list:
    is_found_word = False
    
    for word in report.title.split(' '):
        if ds_filtered[ds['word'] == word].shape[0] > 0:
            in_title_counter += 1
            is_found_word = True
            break

    if not is_found_word:
        not_found_reports_list.append(report)


In [ ]:
len(not_found_reports_list)

In [5]:
from transformers import BertTokenizer
bert_cased_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [6]:
from transformers import BertConfig, BertForPreTraining
config = BertConfig()
model = BertForPreTraining(config)

In [7]:
from transformers import TextDatasetForNextSentencePrediction
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=bert_cased_tokenizer,
    file_path="./preprocessed.txt",
    block_size = 256
)

C:\Users\saif8\miniconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:362: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= "./output",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_gpu_train_batch_size= 16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()
trainer.save_model("./model")

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
No module named 'pandas.api'

In [9]:
import pandas.api

ModuleNotFoundError: No module named 'pandas.api'

In [2]:
import numpy as np
arr = np.array([1, 2, 3, 4, 5])

In [1]:
import pandas as pd
import Reports
report_list = Reports.get_all_reports('/scratch/users/k21148846/report_summary_extracted_data.jsonl')

In [2]:
import re
paragraph_list = []
paragraph_info_list = []
for report in report_list:
    for page in report.pages_list:
        sentence_list = []
        for block in page.block_list:
            # use text_str
            # remove all numbers
            # ignore if no full stop or comma
            # split by full stop
            # filtered_text = re.sub(r'\d+', '', block.text_str)
            sentence_list.append(block.text_str.strip())
        
        paragraph = ' '.join(sentence_list)
        paragraph_length = len(paragraph.split())
        if paragraph_length < 20:
            continue
        paragraph_list.append(paragraph)
        paragraph_info_list.append((page.block_list[0].page_count, report.url))

In [4]:
from math import ceil
from copy import deepcopy

MAX_SENTENCE_LENGTH = 256
indices_to_pop = []
for i, p in enumerate(deepcopy(paragraph_list)):
    paragraph = p.split()
    length = len(paragraph)
    if i % 100000 == 0:
        print(i, len(paragraph_list))
    if length <= MAX_SENTENCE_LENGTH:
        continue
    
    n_splits = ceil(length/MAX_SENTENCE_LENGTH)
    split_paragraphs = [' '.join(paragraph[s*MAX_SENTENCE_LENGTH:(s*MAX_SENTENCE_LENGTH)+MAX_SENTENCE_LENGTH]) for s in range(n_splits)]
    split_paragraph_infos = [paragraph_info_list[i] for _ in range(n_splits)]
        
    indices_to_pop.append(i)
    
    paragraph_list.extend(split_paragraphs)
    paragraph_info_list.extend(split_paragraph_infos)
# need to reverse first so indices dont get messed up in the original array
for i in indices_to_pop[::-1]:
    paragraph_list.pop(i)
    paragraph_info_list.pop(i)

print('Final length: ', len(paragraph_list))

0 609763
100000 773000
200000 942289
300000 1111212
400000 1278384
500000 1443869
600000 1610487
Final length:  1208546


In [7]:
import pickle

with open('/scratch/users/k21148846/docs_NO_ID.pkl', 'wb') as handle:
    pickle.dump((paragraph_list, paragraph_info_list), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
len(paragraph_info_list)

1208546

In [7]:
for i, p in enumerate(paragraph_list):
    if p == '':
        paragraph_list.pop(i)

In [6]:
import pickle

with open('../notebooks_testing/docs_NO_ID.pkl', 'rb') as fp:
    docs = pickle.load(fp)

with open('../notebooks_testing/results.pkl', 'rb') as handle:
    results = pickle.load(handle)

import pandas as pd
result_df = pd.DataFrame(results)
docs_df = pd.DataFrame({'docs': docs})
df = pd.concat([docs_df, result_df], axis=1)
df = df.sample(frac=1).reset_index(drop=True).loc[:,~df.columns.duplicated()].copy() # shuffle

del docs, results, result_df, docs_df

In [7]:
df.loc[(df['score'] < 0.7) & (df['label'] == 'Governance'), 'label'] = 'None'
df.loc[(df['score'] < 0.9) & (df['label'] == 'Social'), 'label'] = 'None' 
df.loc[(df['score'] < 0.9) & (df['label'] == 'Environmental'), 'label'] = 'None' 

In [8]:
import pickle

with open('docs_NO_ID.pkl', 'wb') as handle:
    pickle.dump(paragraph_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
# getting terms and remove the quality value
f = open('./AutoPhrase.txt', 'r', encoding='utf-8')
lines = f.readlines()
f.close()

In [2]:
import string
from nltk.tokenize import word_tokenize

printable = set(string.printable)

terms = []
for line in lines:
    splitted_line = line.split('\t')
    terms.append(splitted_line[1])

f = open('./terms.txt', 'w', encoding='utf-8')
for i, term in enumerate(terms):
    ascii_term = bytes(term, 'ascii', errors='ignore').decode()
    filtered_term = ''.join((filter(lambda x: x in printable, ascii_term)))
    f.write(filtered_term)
    terms[i] = term[:-1]

In [45]:
import string

esg_topic = 'Governance'
f = open("../dataset/terms_" + esg_topic + ".txt", 'r', encoding='utf-8')
terms = [term[:-1] for term in f.readlines()]
f.close()

In [46]:
terms_small = terms[:110000]
terms_multiwords = []

for term in terms_small:
    split_term = term.split()
    if len(split_term) > 1:
        terms_multiwords.append(' '.join(split_term))

In [47]:
def replace_with_index(s, newstring, start_index, end_index):
    return s[:start_index] + newstring + s[end_index:]

In [48]:
# terms = [' '.join(term.split('_')) for term in terms]

In [49]:
import re

docs = df[df['label']==esg_topic]['docs'].to_list()

matching_regex = '|'.join(terms_multiwords)
# matching_regex = matching_regex.replace('||', '|')
pattern = re.compile(matching_regex)

for i, doc in enumerate(docs):
    doc_lowered = doc.lower()
    matched = pattern.match(doc_lowered)
    while bool(matched):
        start_index, end_index = matched.start(), matched.end()
        word_with_underscore = matched.group(0).replace(' ', '_')

        doc_lowered = replace_with_index(doc_lowered, word_with_underscore, start_index, end_index)

        matched = pattern.match(doc_lowered)

    docs[i] = doc_lowered
    if i % 100000 == 0:
        print(i)


0


In [50]:
from nltk.tokenize import word_tokenize

tokenized_docs = []
for i, doc in enumerate(docs):
    tokenized_docs.append(' '.join(word_tokenize(bytes(doc, 'ascii', errors='ignore').decode())))
    if i % 100000 == 0:
        print(i)


0


In [51]:
printable = set(string.printable)

f = open("../dataset/terms_" + esg_topic + ".txt", 'w', encoding='utf-8')
for i, term in enumerate(terms):
    ascii_term = bytes(term, 'ascii', errors='ignore').decode()
    filtered_term = ''.join((filter(lambda x: x in printable, ascii_term)))
    f.write('_'.join(filtered_term.split()) + '\n')
    terms[i] = term[:-1]
f.close()

In [52]:
with open("../dataset/docs_" + esg_topic + ".txt", 'w') as f:
    for doc in tokenized_docs:
        f.write(doc)
        f.write('\n')

In [86]:
backup.group(0)

'external initiatives'

In [17]:
matching_regex.replace('||', '|')

True

In [38]:
'||' in matching_regex

False

In [6]:
matching_regex[-100:-1]

'\\ ebus|\\ sourcing tool|\\ seurujoki|\\ siniscola|\\ energieeffizienz|\\ bowles|\\ svetloye|\\ prefecture’'

In [24]:
import string
printable = set(string.printable)

f = open('./docs.txt', 'r', encoding='utf-8')
lines = f.readlines()
f.close()

f = open('./docs.txt', 'w', encoding='utf-8')
for line in lines:
    ascii_line = bytes(line, 'ascii', errors='ignore').decode()
    filtered_line = ''.join((filter(lambda x: x in printable, ascii_line)))
    f.write(filtered_line)
f.close()

In [2]:
import pandas as pd
df = pd.read_csv('crude-oil-60.csv')
df.head()

,Month,Price,Change
0,May-18,269.67,-
1,Jun-18,264.35,-1.97%
2,Jul-18,266.88,0.96%
3,Aug-18,261.04,-2.19%
4,Sep-18,276.76,6.02%


In [7]:
# avg change from 44 to 59
import matplotlib as plt

print((302.83 / 267.62 - 1) * 100) 

13.156714744787369


In [11]:
print((df['Price'].iloc[49] / df['Price'].iloc[43] - 1) * 100)

0.6028323742620132


In [14]:
find_change_percent = lambda x, y: ((x/y) - 1) * 100
print(find_change_percent(df['Price'].iloc[46], df['Price'].iloc[43]))

54.244824751513356
